In [17]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.data_processing.read import read_device_status_from_zip, parse_date_columns, read_all_device_status
from src.configurations import Configuration


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# For an individual
zip_id = '00221634'
config = Configuration()
file = config.data_dir + '\\' +zip_id + '.zip'
read_record = read_device_status_from_zip(file,config)

time_cols = [c for c in config.time_cols() if c in read_record.df.columns]

df = read_record.df[time_cols].melt()
df['datetime'] = parse_date_columns(config.treat_timezone, df['value'])
df['timezone'] = df['datetime'].dt.tz
df['timezone'].unique()


00221634


array([datetime.timezone.utc], dtype=object)

In [6]:
# Now applied across all zip files
all_records = read_all_device_status(config)

00221634
00309157
00897741
01177138
01352464
01739655
01884126
01919652
01949240
02033176
02050717
02199852
02611986
03403352
03572116
04762925
05274556
05582191
07613176
07886752
10540336
12689381
13029224
13484299
13708515
13783771
14092221
14470046
15558575
15634563
16553776
16975609
17161370
18001564
18991425
19626656
20216809
20396154
20649783
20656313
20777653
21946407
22961398
23340371
23428091
23711486
23769130
23863411
24110807
24448124
24587372
25401109
25692073
26691577
26856617
27526291
27553507
27700103
27819368
28176124
28608066
28756888
28761103
28768536
28823146
32407882
32635618
32997134
33324736
33470634
33831564
33962890
33999544
34148224
35187603
35533061
35719805
37764532
37875431
37948668
37998755
38110191
39038570
39079816
39182506
39819048
39901815
39986716
40237051
40634871
40997757
41131654
41663654
42052178
42360672
43589707
45025419
45120081
46253612
47323535
47631371
47750728
47971065
48509634
48540630
49141524
49182092
49551394
49623237
49796612
50311906
5

In [14]:
def extract_timezone(read_record, config):
    try:
        time_cols = [c for c in config.time_cols() if c in read_record.df.columns]
    except AttributeError:
        print(f'{read_record.zip_id} has no df')
        return

    df = read_record.df[time_cols].melt()
    df['datetime'] = parse_date_columns(config.treat_timezone, df['value'])
    df['timezone'] = df['datetime'].dt.tz
    return df['timezone'].unique()

timezones_list = []
for rr in all_records:
    timezones_list.append((rr.zip_id, extract_timezone(rr, config)))


01177138
01739655
01884126
01919652
01949240
03403352
07886752
10540336
12689381
13708515
14470046
16553776
18001564
20656313
20777653
23340371
23711486
23769130
23863411
24110807
24448124
25401109
26691577
27526291
27553507
28756888
28761103
28823146
32635618
33324736
33470634
33831564
33962890
33999544
34148224
39038570
39079816
39819048
39901815
42052178
45120081
46253612
47631371
47971065
48509634
48540630
49141524
49623237
50311906
50729763
51359431
51622121
51848391
51968000
52345968
53013068
53486744
54165023
54288462
58680324
58845192
61179686
61403732
62371014
63937017
66773091
68659836
69739062
71422852
74241108
80373992
80625186
80901258
81195091
81518757
81742040
84109428
84181797
85078542
86298207
87635714
89032650
89727223
92204064
92229310
92762655
93230373
94098083
94719978
94875538
95024302
95614431
97099623
97385078
98120605
98340749


In [18]:
df = pd.DataFrame(timezones_list, columns=['zip_id', 'timezone'])


,zip_id,timezone
0,00221634,[UTC]
1,00309157,[UTC]
2,00897741,[UTC]
3,01177138,None
4,01352464,[UTC]


In [22]:
print(f'{len(timezones_list)} zip files read')
print(f'Of which there are {df.isna().sum().sum()} that returned a null for timezone')
print(f'The following are the list of different timezones:')
print(df['timezone'].astype(str).unique())


229 zip files read
Of which there are 96 that returned a null for timezone
The following are the list of different timezones:
['[datetime.timezone.utc]' 'None']
